# Assignment 2 Workbook

## Question 1: SVM Primal Form

Please implement the training and testing algorithms of soft-margin Linear Support Vector Machine in its primal form, that is,

$$\min_{\mathbf{w},b,\{\xi_i\}} \frac{1}{2} \|\mathbf{w}\|_2^2 + \frac{C}{N} \sum_{i=1}^N \xi_i \nonumber \\ s.t.~~ y_i (\mathbf{w}^\top \mathbf{x}_i + b) \ge 1 - \xi_i, ~~\forall i \nonumber \\ \xi_i \ge 0 \nonumber$$
Use CVXPY in your implementation strictly following the format given in this Notebook, and supplying missing code in the indicated space.


In [1]:
# import required libraries
import cvxpy as cp
import pandas as pd
import numpy as np

In [2]:
# get training dataset
train = "train.csv"
df = pd.read_csv(train, header=None)
X_train = df[:2000].iloc[:, 1:].to_numpy()
Y_train = df[:2000].iloc[:, 0].replace(0, -1).to_numpy()

In [3]:
# get test dataset
test = "test.csv"
df = pd.read_csv(test, header=None)
X_test = df.iloc[:1000, 1:].to_numpy()
Y_test = df.iloc[:1000, 0].replace(0, -1).to_numpy()

In [4]:
print("X train shape",X_train.shape)
print("y train shape",Y_train.shape)
print("X test shape",X_test.shape)
print("y test shape",Y_test.shape)

X train shape (2000, 200)
y train shape (2000,)
X test shape (1000, 200)
y test shape (1000,)


In [12]:
# train linear svm in primal form
def svm_train_primal(data_train, label_train, regularisation_para_c):
    X, Y = data_train, label_train
    n_samples, m_features = np.shape(X)
    
    W_value = 0
    b_value = 0
    slack_var_value = 0

# ====================== YOUR CODE HERE ======================  
# DO NOT use any other import statements for this question

# ================================================================
    
    # setting the parameter
    W = cp.Variable(m_features)
    b = cp.Variable()
    slack_var = cp.Variable(n_samples)
    
    # objective: Hinge loss functions
    obj = cp.Minimize(
        0.5 * cp.norm(W, 2)**2 + regularisation_para_c / n_samples * cp.sum(slack_var)
    )
    
    # constraints: SVM primal form
    cons1 = [Y[i] * (W.T @ X[i] + b) >= 1 - slack_var[i] for i in range(n_samples)]
    cons2 = [slack_var >= 0]
    cons =  cons1 + cons2
    
    # Form and solve problem
    prob = cp.Problem(obj, cons)
    prob.solve() # Returns the optimal value
    
    # print the problem info - remove it after
    print("status:", prob.status)
    print("optimal value", prob.value)
    print("optimal var", X, Y)
    
    # Extract the values - a numpy ndarray
    W_value = W.value
    b_value = b.value
    slack_var_value = slack_var.value

    return [W_value, b_value, slack_var_value]

# train primal model
c = 100
model_primal = svm_train_primal(X_train, Y_train, c)

# output svm primal form solutions
W = model_primal[0]
b = model_primal[1]

print('Please copy the folowing result to Question 1 "sum(W) = ')
print(np.round(np.sum(W),2))
print('Please copy the folowing result to Question 1 "b = "')
print(np.round(b,2))

status: optimal
optimal value 4.424257612089827
optimal var [[-0.36 -0.91 -0.99 ...  0.3   2.44 -1.26]
 [-1.4  -1.9   0.09 ... -0.2  -0.92 -0.46]
 [-0.43  1.45 -0.68 ...  0.12  0.01 -0.56]
 ...
 [ 1.06  1.93 -0.17 ...  1.17 -0.4  -0.97]
 [-1.02 -0.81  1.01 ... -0.97 -0.34 -0.8 ]
 [-0.75 -1.09 -0.14 ...  1.31 -0.01  1.15]] [-1.  1.  1. ... -1. -1.  1.]
Please copy the folowing result to Question 1 "sum(W) = 
0.79
Please copy the folowing result to Question 1 "b = "
1.75


In [13]:
# predict accuracy of svm model on test dataset
def svm_predict(data_test, label_test, svm_model):
    
    acc = 0.0
    
# ====================== YOUR CODE HERE ======================  
# DO NOT use any other import statements for this question

# ==========================================================
    # Convert test data and labels to NumPy arrays
    X_test = np.array(data_test)
    Y_test = np.array(label_test)
    
    # Retrieve the model parameters
    W_value = svm_model[0]
    b_value = svm_model[1]
    
    # Compute the predictions
    pred = np.sign(X_test @ W_value + b_value)
    
    # Compute the accuracy
    acc= np.mean(pred == Y_test)
    
    return acc

# predict and output primal accuracy
accuracy = svm_predict(X_test, Y_test, model_primal)
print('Please copy the folowing result line to Question 1 "Accuracy = )"')
print(np.round(accuracy,2))

Please copy the folowing result line to Question 1 "Accuracy = )"
0.96


## Question 2: SVM Dual Form

Please implement the training and testing algorithms of soft-margin Linear Support Vector Machine in its **dual** form, that is,

$$\max_{\alpha_i}\sum_i \alpha_i - \frac{1}{2}\sum_i \sum_j \alpha_i \alpha_j y_i y_j <\mathbf{x}_i, \mathbf{x}_j> \nonumber \\ s.t. ~~~ 0 \le \alpha_i \le \frac{C}{N} \nonumber \\ ~~~ \sum_i \alpha_i y_i = 0 \nonumber$$

Use CVXPY in your implementation strictly following the format given in this Notebook, and supplying missing code in the indicated space.


In [2]:
# import required libraries
import cvxpy as cp
import pandas as pd
import numpy as np

In [21]:
# get training dataset
train = "train.csv"
df = pd.read_csv(train, header=None)
X_train = df[:2000].iloc[:, 1:].to_numpy()
Y_train = df[:2000].iloc[:, 0].replace(0, -1).to_numpy()

In [20]:
# get test dataset
test = "test.csv"
df = pd.read_csv(test, header=None)
X_test = df.iloc[:1000, 1:].to_numpy()
Y_test = df.iloc[:1000, 0].replace(0, -1).to_numpy()

In [29]:
# train linear svm in dual form
def svm_train_dual(data_train, label_train, regularisation_para_c):
    
    alphas = 0

# ====================== YOUR CODE HERE ======================  
# DO NOT use any other import statements for this question
    
# ===========================================================

    # setting the parameter
    X, Y = data_train, label_train
    # Y = label_train
    n_samples, m_features = X.shape
    
    alphas = cp.Variable(n_samples)
    # Y = Y.reshape(-1)
    
    # objective: dual  matrix 
    print("X.shape", X.shape)
    print("Y.shape", Y.shape)
    # Gram matrix
    # K = X @ X.T
    # Q matrix
    # Q = Y @ Y.T * K
    # P = cp.matmul(Y, Y.T) @ cp.matmul(X, X.T)
    # results = alphas.T @ P @ alphas
    # P = np.dot(cp.matmul(Y, X),cp.matmul(Y, X).T)
    # P = cp.matmul(Y, Y.T) * X @ X.T
    # P = cp.Constant(P)
    quadratic_part = 0.5 * cp.sum_squares(cp.multiply(alphas, Y) @ X)
    obj = cp.Maximize(cp.sum(alphas) - quadratic_part)
   
    # # # constraints: SVM dual form
    cons = [
        alphas >= 0,
        alphas <= regularisation_para_c / n_samples,
        cp.sum(cp.multiply(alphas, label_train)) == 0
    ]
    
    # # # Form and solve problem
    prob = cp.Problem(obj, cons)
    prob.solve(solver=cp.SCS,verbose=True) # Returns the optimal value
    
    # # print the problem info - remove it after
    # print("status:", prob.status)
    # print("optimal value", prob.value)
    
    # # Extract the values - a numpy ndarray
    alphas_value = alphas.value   

    # # return svm dual model alphas
    return alphas_value
    # return obj

# train dual model
c = 100
alphas = svm_train_dual(X_train, Y_train, c)
print(alphas)
# output svm dual form solutions
print('Please copy the folowing result to Question 2 "Sum of alphas = )"')
print(np.round(np.sum(alphas),2))

X.shape (2000, 200)
Y.shape (2000,)
                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Aug 04 01:00:01 AM: Your problem has 2000 variables, 3 constraints, and 0 parameters.
(CVXPY) Aug 04 01:00:01 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Aug 04 01:00:01 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Aug 04 01:00:01 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Aug 04 01:00:01 AM: Compiling problem (target solver=SCS).
(CVXPY) Aug 04 01:00:01 AM: Reduction chain: FlipObjective 

In [30]:
# obtain primal w*, b* from dual solution
def find_model_params_from_dual(data, label, alphas, regularisation_para_c):
    
    # this value is used to compare values generated by CVXPYY to zero.
    # for example, instead of 
    # if var > 0, we use: if var > zero_threshold
    zero_threshold = 0.0001
    w_dual = 0
    b_dual = 0

    # b is scalar , but may be sliglty diffrent for each Y, calculate b for each label and return mean of them
# ====================== YOUR CODE HERE ======================  
# DO NOT use any other import statements for this question
# Note: b_dual is a scalar, but maybe different for each label, therefore calculate the mean

    # Compute w as a weighted sum of the data points
    # print("alphas shape", alphas.shape)
    # print("label shape", label.shape)
    # print("data shape", data.shape)
    w_dual = (alphas * label) @ data
    
    # Identify support vectors (those with 0 < alpha < C)
    sv_idx = (alphas > zero_threshold) & (alphas < regularisation_para_c - zero_threshold)
    
    # Compute b using support vectors
    support_vector_data = data[sv_idx]
    support_vector_labels = label[sv_idx]
    b_dual_values = support_vector_labels - support_vector_data @ w_dual
    b_dual = np.mean(b_dual_values)
    
# =========================================================================
    # print("sv_idx: ",sv_idx)
    # print("w_dual: ",w_dual)
    # print("b_dual: ",b_dual)
    return w_dual, b_dual

# output reconstructed w* and b* from svm dual problem
c = 100
model_dual = find_model_params_from_dual(X_train, Y_train, alphas, c)

# predict and output accuracy of dual model
accuracy = svm_predict(X_test, Y_test, model_dual)
print('Please copy the folowing result to Question 2 "sum(W) = "')
print(np.round(np.sum(model_dual[0]),2))
print('Please copy the folowing result to Question 2 "b = "')
print(np.round(model_dual[1],2))
print('Please copy the folowing result to Question 2 "accuracy = "')
print(np.round(accuracy,2))

Please copy the folowing result to Question 2 "sum(W) = "
0.79
Please copy the folowing result to Question 2 "b = "
1.73
Please copy the folowing result to Question 2 "accuracy = "
0.96


## Question 3: PCA

1. Perform PCA on the dataset to reduce each sample into a 10-dimensional feature vector.
2. For easy verification, show the sum of covariance matrix.

=========================================================================
- Implementing PCA algorithm.
    - Start
        - Input: n no. of samples as matrix $X$ of $n$ rows and $k$ columns.
        - Calculate the mean for each column. $$mean = \frac {1}{n} \sum \limits _{i=1} ^{n}X_{ij}$$
        - Calculate the centralised matrix $X_C$ and covariance matrix $C$. $$X_C=X-mean$$ $$C = \frac {1}{n}(X_C)^TX_C$$
        - Calculate the eigenvalues and eigenvectors using convariance matrix.
        - Select top x principal components - which are eigen vector corresponding to top x eigen values. Construct matrix $P$.
    - End
    
- Transforming the the data using the principal components (matrix $P$) obtained using the PCA algorithm. $$Transformed \: Data = XP$$
- Calculating the covariance matrix of the transformed data by first centralising it(mean subtracted) and then obtaining the covariance matrix.

In [13]:
# import required libraries
import cvxpy as cp
import pandas as pd
import numpy as np

In [14]:
# get training dataset
train = "train.csv"
df = pd.read_csv(train, header=None)
X = df[:2000].iloc[:, 1:].to_numpy()

In [15]:
# Selecting top 10 Principal components
no_of_components = 10

covariance_matrix_X = 0
covariance_matrix_X_transformed = 0

# ====================== YOUR CODE HERE ======================  
# DO NOT use any other import statements for this question
def PCA(X, num_component):
    
    ## origin dataset
    # Input
    
    # Center the data: Calculate the mean of each column then subtracting the mean
    x_center = X - np.mean(X, axis = 0)
    
    # Calculate the covariance matrix
    x_cov = x_center.T @ x_center / x_center.shape[0]
    
    # Calculate the eigenvalues and eigenvectors using convariance matrix.
    ei_value, ei_vec = np.linalg.eigh(x_cov)
    
    # Sort the eigenvectors based on the eigenvalues
    idx = np.argsort(ei_value)[::-1]
    ei_vec = ei_vec[:,idx]
    
    # Select top x principal components
    top_p_components = ei_vec[:,:num_component]
    
    ## tranformed dataset
    # Input
    x_tran = X @ top_p_components
    
    # Center the data: Calculate the mean of each column then subtracting the mean
    x_tran_center = x_tran - np.mean(x_tran, axis = 0)
    
    # Calculate the covariance matrix
    x_tran_cov = x_tran_center.T @ x_tran_center / x_tran_center.shape[0]
    
    return [x_cov, x_tran_cov]
    
# Get the covariance matrix both before and after tranformation
covariance_matrix_X, covariance_matrix_X_transformed = PCA(X, no_of_components)

# ==========================================================================

sum_cov_X = np.sum(covariance_matrix_X)
sum_cov_X_transformed = np.sum(covariance_matrix_X_transformed)

print('Please copy the folowing result to Question 3 "Cov X = )"')
print(np.round(np.sum(sum_cov_X),2))
print('Please copy the folowing result to Question 3 "Cov X_transformed = )"')
print(np.round(np.round(sum_cov_X_transformed,2)))


Please copy the folowing result to Question 3 "Cov X = )"
263.04
Please copy the folowing result to Question 3 "Cov X_transformed = )"
57.0


## Question 4: Kernel k-Means

In this question you will implement kernel k-means with RBF-kernel, using SpectralClustering from sklearn to reduce the programming efford of the full implementation. 
Please implement kernel k-means algorithm with RBF-kernel, that is:

$$(\mathbf{x}_i,\mathbf{x}_j) = \exp(\frac{-\|\mathbf{x}_i - \mathbf{x}_j\|_2^2}{2\sigma^2})$$

For the $2\sigma^2$ parameter, instead of a constant value, please use mean pairwise squared distance between the datapoints, that is:

$$2\sigma^2 = \frac{1}{N^2} \sum_{i=1}^{N}\sum_{j=1}^{N} \| \mathbf{x}_i -\mathbf{x}_j\|_2^2$$ 

In order to calculate paiwise distance matrix, please use scipy.spatial distance function, which is alread pre-loaded in the code. <br>
Please check the documentation of the SpectralClustering function to use precomputed arrinity matrix, which you are going to supply with the above specification.

In [16]:
import pandas as pd
import numpy as np
from sklearn.cluster import SpectralClustering
from scipy.spatial import distance

df = pd.read_csv('train.csv', header = None)
X = df.iloc[:1000,1:].to_numpy(copy=True)
Y = df.iloc[:1000,:1].to_numpy(copy=True)

In [17]:
n_clusters = 10
centroids_rbf = np.array(n_clusters)

centroids_rbf = []

# ====================== YOUR CODE HERE ======================  
# DO NOT use any other import statements for this question
# use random_state=0 in the SpectralClustering function to make the results reproducible.

# calculate pairwise distance
pw_sq_dist = distance.squareform(distance.pdist(X,'sqeuclidean'))

# mean pairwise - 2lambda
mean_pw_sq_dist = 0.5 * np.sum(pw_sq_dist) / (X.shape[0]**2)

# calculate RBF kernal
arrinity_matrix = np.exp(-pw_sq_dist/(mean_pw_sq_dist * 2))

# construct the Spectral Clustering
cluster = SpectralClustering(n_clusters = n_clusters, affinity='precomputed', random_state=0)
label = cluster.fit_predict(arrinity_matrix)

# find the centriod
centroids_rbf = [np.mean(X[label == i], axis=0) for i in range(n_clusters)]
centroids_rbf = np.array(centroids_rbf)

# =================================================================
    
print('Please copy the folowing result to Question 4 "Sum of centroids = )"')
print(np.round(np.sum(centroids_rbf),2))


Please copy the folowing result to Question 4 "Sum of centroids = )"
36.78


## Question 5: Adaboost questions

Answer questions in myUni assignment 2

- [FALSE] Using Adaboost with a stronger weak classifier, for example, weak classifiers that can achieve higher accuracy when they are used individually, can always lead to better accuracy on the test set.
- [TRUE] Assume that the weak learners are a finite set of decision stumps, Adaboost cannot achieve zero training error if the training data is not linearly separable.
- [TRUE] Assume that the weak learners are a finite set of decision stumps, normalising features (scaling each dimension of features to ensure that each dimension has zero mean and unit variance) will not impact the predictive accuracy on the test set.
- [FALSE] Once a weak classifier is picked in a particular round, it will never be chosen in any subsequent round.

## Question 6: Adaboost code correction
The following code shows an incomplete implementation of Adaboost algorithm. The code does not implement the logic specific to Adaboost. Please modify it to make a correct implementation. Modify the code only in sections as indicated


In [18]:
##### do not change this code

# import required libraries
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from math import exp, log as ln

number_of_models = 3

In [19]:
##### do not change this code

# get training dataset
train = "train.csv"
df = pd.read_csv(train, header=None)
X_train = df.iloc[:, 1:].to_numpy()
Y_train = df.iloc[:, 0].replace(0, -1).to_numpy()

In [20]:
##### do not change this code

# get test dataset
test = "test.csv"
df = pd.read_csv(test, header=None)
X_test = df.iloc[:, 1:].to_numpy()
Y_test = df.iloc[:, 0].replace(0, -1).to_numpy()

In [21]:
##### do not change this code

def weak_classifier_train(train_data, train_label, sample_weights):
    # Create a decision stump
    stump = DecisionTreeClassifier(max_depth=1)
    
    # Train the stump using the weighted samples
    stump.fit(train_data, train_label, sample_weight=sample_weights)
    
    # Generate predictions and calculate error
    model_prediction = stump.predict(train_data)
    train_label = np.array(train_label)
    model_prediction = np.array(model_prediction)
    sample_weights = np.array(sample_weights)

    # The following line should work if both train_label and model_prediction are numpy arrays of the same size.
    error = np.sum(sample_weights[train_label != model_prediction])
    
    return stump, error, model_prediction

##### do not change this code

def weak_classifier_prediction(test_data, model):
    
    # The model_param_t is our trained stump, so we use it to make predictions
    return model.predict([test_data])

**You can change the code in the following section only**
**Please clearly comment the purpose of the code that you added or changed.**

In [22]:
def Adaboost_train(train_data, train_label, T):
    # train_data: N x d matrix
    # train_label: N x 1 vector
    # T: the number of weak classifiers in the ensemble

    ensemble_models = []
    alpha_value = [] ## [Adding] - check list to collet alpha value

    N = len(train_label)  # The number of samples

    # Initialize weight distribution for each sample to be evenly distributed
    D = np.array([1 / N] * N)

    # For each weak classifier, do the following
    for t in range(T):

        # weak_classifier_train takes the training data and labels as well as the sample weight distribution
        # returns the model parameters and the error
        model, error, model_prediction = weak_classifier_train(train_data, train_label, D) 
        
        ## [Adding] - calculate alpha from error
        alpha = 0.5 * np.log((1 - error) / error)
        
        ## [Adding] - update new weight + normalization
        D = D * np.exp(-alpha * train_label * model_prediction) / np.sum(D)
        
        # definition of model
        ensemble_models.append(model)
        alpha_value.append(alpha) ## [Adding] - append list to collet alpha value

    return ensemble_models, alpha_value


def Adaboost_test(test_data, ensemble_models, alpha_value): ## [Adding] alpha value to adjust the H = sum (alpha x pred)
    # test_data: n x d
    predictions = []

    for i in range(len(test_data)):
        decision_ensemble = 0

        # For each model in the ensemble models, do the following
        for k in range(len(ensemble_models)):

            # Make a prediction on the classification of the sample
            # prediction returns 1 or -1 prediction from the weak classifier
            prediction = weak_classifier_prediction(test_data[i], ensemble_models[k])

            # Add this classification 
            decision_ensemble += alpha_value[k] * prediction ## [Adding] alpha value

        # If more models predicted it as +1 class
        if decision_ensemble > 0:
            prediction = 1
        # Else, more models predicted it as -1 class
        else:
            prediction = -1

        predictions.append(prediction)

    return predictions

# predict and output accuracy

ensemble_models, alpha_value = Adaboost_train(X_train, Y_train, number_of_models)
predicted_labels = Adaboost_test(X_test, ensemble_models, alpha_value)

In [23]:
######## do not change this code ############

# Calculate accuracy
accuracy = accuracy_score(Y_test, predicted_labels)

print('Please copy the following result line to Question 6 "Accuracy = )"')
print(np.round(accuracy,3))

Please copy the following result line to Question 6 "Accuracy = )"
0.981


## Question 7: Deep Learning

Answer this question in myUni assignment 2

## derive the gradient for tanh activation function

01 Show the derivation following the function below

\begin{align*}
\frac{\partial{E( {w})}}{\partial{ {w}}} & = \frac{\partial{(\frac{1}{2}\sum(t - o)^{2})}}{\partial{ {w}}} \\
& = \frac{\partial{(\frac{1}{2}\sum(t - \tanh(xw))^{2})}}{\partial{ {w}}} \\
& = \frac{1}{2} * \sum(t - \tanh(xw)) * 2 * \frac{\partial{ (t - \tanh(xw))}}{\partial{ {w}}}\\
& = - \sum (t - \tanh(xw)) \frac{\partial{\tanh(xw)}}{\partial{ {w}}} \\
& = - \sum (t - \tanh(xw)) (1 - \tanh^{2}(xw)) \frac{\partial{(xw)}}{\partial{ {w}}} \\
& = - \sum (t - \tanh(xw)) (1 - \tanh^{2}(xw)) x \\
\end{align*}

given just one example of the following data 
\begin{align*}
x & = 1.0 \\
w & = 0.3 \\
t & = 1
\end{align*}

02 Calculate the output of the activation function

\begin{align*}
f(xw) & = x * W \\
& = 1.0 * 0.3 \\
& = 0.3 \\
output & = tanh(xw) \\
& = \frac{1 - e^{-2x}}{1 + e^{-2x}} \\
& = \frac{1 - e^{-2 * 0.3}}{1 + e^{-2 * 0.3}} \\
& = \frac{1 - e^{-0.6}}{1 + e^{-0.6}} \\
& = 0.2913 \approx 0.29
\end{align*}

03 Calculate the gradient value

\begin{align*}
\frac{\partial{E( {w})}}{\partial{ {w}}} & = - \sum (t - \tanh(xw)) (1 - \tanh^{2}(xw)) x \\
& = -(1 - 0.29)) (1 - 0.29^{2}) * 1.0 \\
& = -0.6502 \approx -0.65

\end{align*}